In [4]:
import sys
sys.path.append('../scdv')

In [5]:
import nltk
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
from nltk import word_tokenize
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline

In [6]:
from scdv import SCDV
from baseline import BaselineEmbedding

In [13]:
scdv_model = SCDV.load("../saved_models/bbc_word2vec_sg_5_100.pkl")
baseline_model = SCDV.load("../saved_models/baseline_bbc_word2vec_sg_100.pkl")

In [14]:
data_path = "../data/bbc/all/"

documents = list()
p = Path(data_path)
files = list(p.glob("**/*.txt"))
for file in tqdm(files):
    try:
        with open(file, "r", encoding='utf8') as f:
            text = f.read().strip()
    except:
        pass
    documents.append(text)
    
total_documents = len(documents)
document_words = [word_tokenize(document) for document in tqdm(documents)]

  0%|          | 0/2225 [00:00<?, ?it/s]

  0%|          | 0/2225 [00:00<?, ?it/s]

In [15]:
def get_qd_score(model, query_words, document_idx, query_unigram_probabilities, document_unigram_probabilities, document_vectors):
    _sum = 0
    for word in query_words:
        _sum += query_unigram_probabilities[word] * get_probability_wd(word, document_idx, document_unigram_probabilities, model, document_vectors)
    return _sum

def get_scores(model, query_vector, document_vectors, query_words, document_idx, query_unigram_probabilities, document_unigram_probabilities, lambda_=0.5):
    document_vector = document_vectors[document_idx]
    similarity = model.similarity(query_vector, document_vector)
    score_qd = get_qd_score(model, query_words, document_idx, query_unigram_probabilities, document_unigram_probabilities, document_vectors)
    score_pv = (1 - lambda_) * score_qd + lambda_ * similarity
    return score_pv, score_qd

def get_probability_pv(word, document_idx, model, document_vectors):
    word_vector = model.get_word_vector(word)
    document_vector = document_vectors[document_idx]
    similarity = model.similarity(word_vector, document_vector)
    similarity_exponent = np.exp(similarity)
    _sum = 0
    for word in document_words[document_idx]: # model.vocabulary: # is this model vocabulary?
        _sum += np.exp(model.similarity(document_vector, model.get_word_vector(word)))
    probability_pv = similarity_exponent / _sum
    return probability_pv

def get_probability_wd(word, document_idx, document_unigram_probabilities, model, document_vectors, lambda_=0.5):
    probability_lm = document_unigram_probabilities[document_idx].get(word, 0)
    return (1 - lambda_) * probability_lm + lambda_ * get_probability_pv(word, document_idx, model, document_vectors)

def make_sparse_document_vectors(document_vectors, p=0.5):
    ndim = document_vectors.shape[1]
    min_ndim = list()
    max_mdin = list()
    for i in range(ndim):
        min_ndim.append(np.min(document_vectors[:, i]))
        max_mdin.append(np.max(document_vectors[:, i]))
    a_min = np.mean(min_ndim)
    a_max = np.mean(max_mdin)
    t = (np.abs(a_min) + np.abs(a_max)) / 2
    pt = p * t
    document_vectors[np.abs(document_vectors) < pt] = 0
    return document_vectors

In [16]:
scdv_document_vectors = [scdv_model.get_document_vector(word_tokenize(document)) for document in tqdm(documents)]
scdv_document_vectors = np.asarray(scdv_document_vectors)
scdv_document_vectors = make_sparse_document_vectors(scdv_document_vectors)

baseline_document_vectors = [baseline_model.get_document_vector(word_tokenize(document)) for document in tqdm(documents)]
baseline_document_vectors = np.asarray(baseline_document_vectors)

  0%|          | 0/2225 [00:00<?, ?it/s]

/home/pes1ug19cs592/SCDV/notebook/../scdv/scdv.py:107: RuntimeWarning: invalid value encountered in true_divide
  vector = vector / np.linalg.norm(vector)


  0%|          | 0/2225 [00:00<?, ?it/s]

In [17]:
document_lm = list()
document_unigram_probabilities = list()
for document in tqdm(document_words):
    unigram_probabilities = dict()
    train, vocab = padded_everygram_pipeline(3, [document])
    lm = MLE(3)
    lm.fit(train, vocab)
    document_lm.append(lm)
    for word in document:
        unigram_probabilities[word] = lm.score(word)
    document_unigram_probabilities.append(unigram_probabilities)

  0%|          | 0/2225 [00:00<?, ?it/s]

In [18]:
def get_query_results(query):
    query_words = word_tokenize(query)
    query_unigram_probabilities = dict()
    train, vocab = padded_everygram_pipeline(1, [query])
    lm = MLE(1)
    lm.fit(train, vocab)
    for word in query_words:
        query_unigram_probabilities[word] = lm.score(word)
        
    query_vector_scdv = np.asarray(scdv_model.get_document_vector(query_words))
    query_vector_baseline = np.asarray(baseline_model.get_document_vector(query_words))
    
    scores_scdv = list()
    scores_baseline = list()
    
    for document_idx in tqdm(range(total_documents)):
        score_lm = document_lm[document_idx].score(query_words[-1], query_words[:-1])
        
        score_pv, score_qd = get_scores(scdv_model, query_vector_scdv, scdv_document_vectors, query_words, document_idx, query_unigram_probabilities, document_unigram_probabilities)
        scores_scdv.append((score_pv, score_qd, score_lm, document_idx))
        
        score_pv, score_qd = get_scores(baseline_model, query_vector_baseline, baseline_document_vectors, query_words, document_idx, query_unigram_probabilities, document_unigram_probabilities)
        scores_baseline.append((score_pv, score_qd, score_lm, document_idx))
        
    scores_scdv.sort(key=lambda x: x[0], reverse=True)
    scores_baseline.sort(key=lambda x: x[0], reverse=True)
    
    scores_scdv = scores_scdv[:3]
    scores_baseline = scores_baseline[:3]
    
    for score in scores_scdv:
        print(f"Similarity: {score[0]}\n\nDocument: {documents[score[-1]]}\n\n")
        
    print("-------------------------------------------------------------------")
    
    for score in scores_baseline:
        print(f"Similarity: {score[0]}\n\nDocument: {documents[score[-1]]}\n\n")

In [19]:
get_query_results("America")

  0%|          | 0/2225 [00:00<?, ?it/s]

Similarity: 0.24556523949082343

Document: Latin America sees strong growth

Latin America's economy grew by 5.5% in 2004, its best performance since 1980, while exports registered their best performance in two decades.

The United Nations' Economic Commission for Latin America and the Caribbean said the region grew by 5.5% this year. The Inter-American Development Bank (IADB) said regional exports reached $445.1bn (£227bn;331bn euros) in 2004. Doubts about the strength of the US recovery and overheating of the Chinese economy do however pose risks for 2005. Both organisations also warned that high oil prices raise the risk of either inflation or recession.

Nevertheless, the Economic Commission for Latin America and the Caribbean (ECLAC) still forecasts growth of 4% for 2005. Strong recovery in some countries, such as Venezuela and Uruguay, boosted the overall performance of the region. ECLAC also said that the six largest Latin American economies (Argentina, Brazil, Chile, Colombia, 